**Further train DeepCell Models with K's Data and Make Predictions**<br>
The model has three heads: inner distance, outer distance, and fgbg. It works on tensorflow 2.7.1.<br>
We first train a model with nucleus data from the Tissuenet V1.0 dataset and save the model. <br>
We then further train the model via deepcell.training.train_model_sample, which allows arbitrary size images and uses window_size to control patch size.
There are a total of 12,574 training instances.

In [1]:
starting_model_path = 'tn1.0_nuclear_20221102.h5' # the model trained with nucleus data from the Tissuenet V1.0 dataset
model_path = 'tn1.0_nuclear_K.h5' # the model trained with nucleus data from the Tissuenet V1.0 dataset
epochs=60 # 7 sec per epoch
label="nuclear_K"

In [2]:
import syotil

import numpy as np
from skimage import io
from matplotlib import pyplot as plt
%matplotlib inline
from timeit import default_timer
import os

import deepcell
from deepcell import image_generators
from deepcell.utils.train_utils import rate_scheduler, get_callbacks, count_gpus
from deepcell_toolbox.utils import resize, tile_image, untile_image
from deepcell_toolbox.deep_watershed import deep_watershed
from deepcell.losses import weighted_categorical_crossentropy
from deepcell.model_zoo.panopticnet import PanopticNet

import tensorflow as tf
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.losses import MSE

print(tf.__version__)
print(deepcell.__version__)

2.7.1
0.11.1


In [3]:
import glob
INPUT_PATH="images/training/"
FILENAMES = glob.glob(INPUT_PATH+"*_img.png")
print(len(FILENAMES))
print(FILENAMES)

7
['images/training/M872956_JML_Position9_CD3_train_img.png', 'images/training/M872956_JML_Position8_CD3_train_img.png', 'images/training/M926910_CFL_Position7_CD3_train_img.png', 'images/training/M926910_CFL_Position13_CD3_train_img.png', 'images/training/M872956_JML_Position8_CD8_train_img.png', 'images/training/M872956_JML_Position10_CD3_train_img.png', 'images/training/M872956_JML_Position8_CD4_train_img.png']


In [4]:
imgs = [io.imread(CURR_IM_NAME)[:,:,0] for CURR_IM_NAME in FILENAMES]
X_train = tf.stack(imgs)
X_train = np.expand_dims(X_train, axis=-1)
print(X_train.shape)

masks = [io.imread(CURR_IM_NAME.replace("img","masks")) for CURR_IM_NAME in FILENAMES]
y_train = tf.stack(masks)
y_train = np.expand_dims(y_train, axis=-1)
print(y_train.shape)

# np.savez("K_training_data", X=X_train, y=y_train) # objects to save need to be key value pairs
# train_dict, test_dict = get_data("K_training_data.npz", test_size=test_size, seed=seed)

2022-11-02 18:38:17.808095: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13832 MB memory:  -> device: 0, name: Tesla V100-PCIE-16GB, pci bus id: 0000:04:00.0, compute capability: 7.0
2022-11-02 18:38:17.809568: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14208 MB memory:  -> device: 1, name: Tesla V100-PCIE-16GB, pci bus id: 0000:06:00.0, compute capability: 7.0
2022-11-02 18:38:17.810771: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 14208 MB memory:  -> device: 2, name: Tesla V100-PCIE-16GB, pci bus id: 0000:08:00.0, compute capability: 7.0


(7, 1040, 1159, 1)
(7, 1040, 1159, 1)


In [5]:
# resize image
image_mpp=1
shape = X_train.shape
scale_factor = image_mpp / 0.65
new_shape = (int(shape[1] * scale_factor),
             int(shape[2] * scale_factor))

X_train = resize(X_train, new_shape, data_format='channels_last')
y_train = resize(y_train, new_shape, data_format='channels_last')
print(X_train.shape)

(1599, 1783)
(7, 1599, 1783, 1)


In [15]:
FILENAMES

['images/training/M872956_JML_Position9_CD3_train_img.png',
 'images/training/M872956_JML_Position8_CD3_train_img.png',
 'images/training/M926910_CFL_Position7_CD3_train_img.png',
 'images/training/M926910_CFL_Position13_CD3_train_img.png',
 'images/training/M872956_JML_Position8_CD8_train_img.png',
 'images/training/M872956_JML_Position10_CD3_train_img.png',
 'images/training/M872956_JML_Position8_CD4_train_img.png']

In [17]:
# write resized images and masks. saved to images/training
import cv2
for i in range(7):
    io.imsave(FILENAMES[i].replace("_img","_resize_img"), X_train[i,:,:,0])    
    io.imsave(FILENAMES[i].replace("_img","_resize_masks"), y_train[i,:,:,0])    

/tmp/ipykernel_31175/3883742908.py:4: UserWarning: images/training/M872956_JML_Position9_CD3_train_resize_img.png is a low contrast image
  io.imsave(FILENAMES[i].replace("_img","_resize_img"), X_train[i,:,:,0])
Lossy conversion from int32 to uint8. Range [0, 254]. Convert image to uint8 prior to saving to suppress this warning.
/tmp/ipykernel_31175/3883742908.py:4: UserWarning: images/training/M872956_JML_Position8_CD3_train_resize_img.png is a low contrast image
  io.imsave(FILENAMES[i].replace("_img","_resize_img"), X_train[i,:,:,0])
Lossy conversion from int32 to uint8. Range [0, 254]. Convert image to uint8 prior to saving to suppress this warning.
/tmp/ipykernel_31175/3883742908.py:4: UserWarning: images/training/M926910_CFL_Position7_CD3_train_resize_img.png is a low contrast image
  io.imsave(FILENAMES[i].replace("_img","_resize_img"), X_train[i,:,:,0])
Lossy conversion from int32 to uint8. Range [0, 254]. Convert image to uint8 prior to saving to suppress this warning.
/tmp/ip

In [20]:
X_train.shape

(7, 1599, 1783, 1)

In [24]:
prediction_model = PanopticNet(
    backbone='resnet50',
    norm_method='whole_image',
    num_semantic_classes=[1, 1], # inner distance, outer distance
    input_shape= (512,512,1)
)

prediction_model.load_weights(starting_model_path, by_name=True)

2022-11-02 18:59:08.777994: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [28]:
# get tile info 
from deepcell.applications import NuclearSegmentation
app1 = NuclearSegmentation(prediction_model)

y, tile_info = app1._tile_input(np.expand_dims(X_train[0,...],0))
print(X_train.shape)
print(y.shape)
print(tile_info)


(7, 1599, 1783, 1)
(20, 512, 512, 1)
{'batches': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'x_starts': [0, 0, 0, 0, 0, 384, 384, 384, 384, 384, 768, 768, 768, 768, 768, 1152, 1152, 1152, 1152, 1152], 'x_ends': [512, 512, 512, 512, 512, 896, 896, 896, 896, 896, 1280, 1280, 1280, 1280, 1280, 1664, 1664, 1664, 1664, 1664], 'y_starts': [0, 384, 768, 1152, 1536, 0, 384, 768, 1152, 1536, 0, 384, 768, 1152, 1536, 0, 384, 768, 1152, 1536], 'y_ends': [512, 896, 1280, 1664, 2048, 512, 896, 1280, 1664, 2048, 512, 896, 1280, 1664, 2048, 512, 896, 1280, 1664, 2048], 'overlaps_x': [(0, 128), (0, 128), (0, 128), (0, 128), (0, 128), (128, 128), (128, 128), (128, 128), (128, 128), (128, 128), (128, 128), (128, 128), (128, 128), (128, 128), (128, 128), (128, 0), (128, 0), (128, 0), (128, 0), (128, 0)], 'overlaps_y': [(0, 128), (128, 128), (128, 128), (128, 128), (128, 0), (0, 128), (128, 128), (128, 128), (128, 128), (128, 0), (0, 128), (128, 128), (128, 128), (128, 128), (128, 0), (

In [6]:
seed=0
min_objects = 2

transforms = ['inner-distance', 'outer-distance', 'fgbg']
transforms_kwargs = {'outer-distance': {'erosion_width': 0}}

# use augmentation for training but not validation
# datagen = image_generators.CroppingDataGenerator(
#     crop_size=(512, 512),
datagen = image_generators.SemanticDataGenerator(
    rotation_range=180,
    fill_mode='reflect',
    zoom_range=(0.75, 1.25),
    horizontal_flip=True,
    vertical_flip=True)

datagen_val = image_generators.CroppingDataGenerator()

batch_size = 4 # 8 causes memory outage

train_data = datagen.flow(
    {'X': X_train, 'y': y_train},
    seed=seed,
    transforms=transforms,
    transforms_kwargs=transforms_kwargs,
    min_objects=min_objects,
    batch_size=batch_size)

val_data = None
# datagen_val.flow(
#     {'X': X_val, 'y': y_val},
#     seed=seed,
#     transforms=transforms,
#     transforms_kwargs=transforms_kwargs,
#     min_objects=min_objects,
#     batch_size=batch_size)

# get number of training and validation instances

cnts_train = [np.max(y_train[i,...]) for i in range(y_train.shape[0])]
print(np.sum(cnts_train)) # total number of training instances

12574


In [7]:
x_tmp, y_tmp=train_data.next()

KeyboardInterrupt: 

In [ ]:
i=5
plt.subplot(1, 2, 1) # row 1, col 2 index 1
io.imshow(X_train[i,:,:,0])
plt.subplot(1, 2, 2) # row 1, col 2 index 1
# tmp = syotil.masks_to_outlines(y_train[i,:,:,0]); io.imshow(tmp)
io.imshow(y_train[i,:,:,0])
plt.show()

In [ ]:
X_train.shape
print(1783/512)
1599/512


**The two cells below define and train the model.** They can be skipped if a trained model will be loaded.

In [ ]:
semantic_classes = [1, 1, 2] # inner distance, outer distance, fgbg

model = PanopticNet(
    backbone='resnet50',
    input_shape=(512,512,1),
    norm_method='whole_image',
    num_semantic_classes=semantic_classes)

lr = 1e-4
optimizer = Adam(lr=lr, clipnorm=0.001)
lr_sched = rate_scheduler(lr=lr, decay=0.99)

# Create a dictionary of losses for each semantic head

def semantic_loss(n_classes):
    def _semantic_loss(y_pred, y_true):
        if n_classes > 1:
            return 0.01 * weighted_categorical_crossentropy(
                y_pred, y_true, n_classes=n_classes)
        return MSE(y_pred, y_true)
    return _semantic_loss

loss = {}

# Give losses for all of the semantic heads
for layer in model.layers:
    if layer.name.startswith('semantic_'):
        n_classes = layer.output_shape[-1]
        loss[layer.name] = semantic_loss(n_classes)
        
model.compile(loss=loss, optimizer=optimizer)

model.load_weights(starting_model_path, by_name=True)

[(layer.name, layer.output_shape) for layer in filter(lambda x: x.name.startswith('semantic_'), model.layers)]

In [ ]:
# fit the model
print('Training on', count_gpus(), 'GPUs.')

train_callbacks = get_callbacks(
    model_path,
    lr_sched=lr_sched,
#     monitor="val_loss",
    monitor='loss', # training loss
    verbose=1)          
            
loss_history = model.fit(
    train_data,
    steps_per_epoch=train_data.y.shape[0] // batch_size,
    epochs=2, 
#     validation_data=val_data,
#     validation_steps=val_data.y.shape[0] // batch_size,
    callbacks=train_callbacks)

model.save_weights(model_path)

<B>Make predictions on Nuclear test dataset.</B> 

In [ ]:
prediction_model = PanopticNet(
    backbone='resnet50',
    norm_method='whole_image',
    num_semantic_classes=[1, 1], # inner distance, outer distance
    input_shape= X_train.shape[1:]
)

prediction_model.load_weights(model_path, by_name=True)

**Make prediction on K's data.**<br>
Using NuclearSegmentation allows setting image_mpp, which has a substantial influence on performance.

In [ ]:
from deepcell.applications import NuclearSegmentation
app = NuclearSegmentation(prediction_model)
[(layer.name, layer.output_shape) for layer in filter(lambda x: x.name.startswith('semantic_'), app.model.layers)]

In [ ]:
import os
print(os.getcwd())
INPUT_PATH="images/test/"
FILENAMES = [f for f in os.listdir("images/training/testimages")]
print(FILENAMES)

In [ ]:
APs={}
for CURR_IM_NAME in FILENAMES:
    im0 = io.imread(os.path.join(INPUT_PATH, CURR_IM_NAME))
    mask_true=io.imread(os.path.join(INPUT_PATH, CURR_IM_NAME.replace("img","masks")))

    x = np.expand_dims(im0, axis=-1)
    x = np.expand_dims(x, axis=0)
    y, tile_info = app._tile_input(x)
    print(x.shape)
    print(y.shape)
    print(tile_info)
    pred = app.predict(y, image_mpp=1)
    prd = app._untile_output(pred, tile_info)
    #io.imshow(prd[0,:,:,0])
    plt.show()
    
    APs[CURR_IM_NAME] = syotil.csi(mask_true, prd[0,:,:,0])# masks may lose one pixel if dimension is odd pixels

APs["mAP"]=np.mean(list(APs.values()))
print(APs)

In [ ]:
import pandas as pd
df = pd.DataFrame([FILENAMES+["mAP"], list(APs.values())])
print(df.transpose())
df.to_csv('images/training/csi_tn_'+label+'.txt', index=False, header=False)

**mAP**<br>
image_mpp default: .<br>
image_mpp=1: .34<br>
image_mpp=2: .<br>